# PyCity Schools Analysis Homework

This is the notebook for the analysis of the district of the schools.  I analyzed first the district, then the summary of schools, and then segmented some data to see top and worst performing schools, and the performance by grade of the students.  Finally there is an analysis by type of school and by size and budget amounts assigned to each school.

Here we have some observable trends based on the data analyzed:

* Of all the schools analyzed, the average in mathematics and the average in reading is similar, however, the average in mathematics is always lower in all than reading average.

* In schools with the highest budget, grades are lower, both in math and in reading. The best in performance are all Charter schools, the worst are all Distric type schools. In the worst the average of mathematics is much lower than that of reading that is regular, in the best schools, the averages between both are similar, being a little higher the reading as I already mentioned.

* There is also an interesting trend, the largest schools in size (> 2000 students) have very low performance, the best have a size less than this number except one.

* Students who have an assigned budget over the amount of 615 lower their grades in both math and reading, and the worsts are those with the highest budget allocations, between 645 and 675.


In [1]:
#Import Dependencies
import pandas as pd

In [2]:
#Reading CSV files
f_schools = pd.read_csv("Resources/schools_complete.csv", encoding="ISO-8859-1")
f_students = pd.read_csv("Resources/students_complete.csv", encoding="ISO-8859-1")

In [3]:
#Merge data in a single data set for use it in the next calculations
school_data_c = pd.merge(f_students, f_schools, how="left", on=["school_name", "school_name"])
school_data_c.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
#1-DISTRICT SUMMARY
#Calculate totals and inital statistics:
#Total number of schools
total_schools = len(school_data_c["school_name"].unique())

#Total number of students
total_students = len(school_data_c["Student ID"].unique())

#Total budget
total_budget = f_schools["budget"].sum()

#Average math score
avg_math_score = f_students["math_score"].mean()

#Average reading score
avg_reading_score = f_students["reading_score"].mean()

#Percentage of math passing score
math_passing_score = (len(school_data_c.loc[school_data_c["math_score"] >= 70, :])/total_students) * 100

#Percentage of reading passing score
read_passing_score = (len(school_data_c.loc[school_data_c["reading_score"] >= 70, :])/total_students) * 100

#Overall passing rate
overall_passing_rate = (math_passing_score + read_passing_score)/2

# Place all of the data found into a summary DataFrame
v

#Formatting data
summary_table["Total Students"] = summary_table["Total Students"].map("{:,}".format)
summary_table["Total Budget"] = summary_table["Total Budget"].map("${:,}".format)
summary_table["% Math passing Score"] = summary_table["% Math passing Score"].map("{:2f} %".format)
summary_table["% Read passing Score"] = summary_table["% Read passing Score"].map("{:2f} %".format)
summary_table["% Overall passing rate"] = summary_table["% Overall passing rate"].map("{:2f} %".format)

summary_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Math passing Score,% Read passing Score,% Overall passing rate
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853 %,85.805463 %,80.393158 %


In [5]:
#2-SCHOOL SUMMARY
#Calculate totals and inital statistics BY SCHOOL:
#Group by school name
grouped_by_school = school_data_c.groupby(['school_name'])
#Total students by school
tot_schools = grouped_by_school["school_name"].count()
schools_summary_table = pd.DataFrame({"Number of students": tot_schools})
#Adding more data
schools_summary_table = pd.merge(schools_summary_table, f_schools, on="school_name")
#Reorganize columns
schools_summary_table = schools_summary_table[["school_name","type","Number of students","budget"]]
#Budget per student
schools_summary_table["Budget per Student"] = schools_summary_table["budget"]/schools_summary_table["Number of students"]

average_data_math = grouped_by_school["math_score"].mean()
average_data_read = grouped_by_school["reading_score"].mean()
schools_averages = pd.DataFrame({"Average math": average_data_math,
                                 "Average read": average_data_read})
#Adding more data
schools_summary_table = pd.merge(schools_summary_table, schools_averages, on="school_name")

# % Passing math 
pass_mat_1 = school_data_c.loc[school_data_c["math_score"] >= 70, :]
pass_mat_2 = pass_mat_1.groupby(['school_name'])
pass_mat_3 = pass_mat_2["school_name"].count()
passing_math = pd.DataFrame({"math score": pass_mat_3})
# % Passing read
pass_read_1 = school_data_c.loc[school_data_c["reading_score"] >= 70, :]
pass_read_2 = pass_read_1.groupby(['school_name'])
pass_read_3 = pass_read_2["school_name"].count()
passing_read = pd.DataFrame({"reading score": pass_read_3})
#Merge results
schools_summary_table = pd.merge(schools_summary_table, passing_math, on="school_name")
schools_summary_table = pd.merge(schools_summary_table, passing_read, on="school_name")
#Calculate % math score and reading score
schools_summary_table["% passing math"] = (schools_summary_table["math score"]/schools_summary_table["Number of students"])* 100
schools_summary_table["% passing reading"] = (schools_summary_table["reading score"]/schools_summary_table["Number of students"])* 100
#Calculate overall %
schools_summary_table["% overall passing rate"] = (schools_summary_table["% passing math"] + schools_summary_table["% passing reading"])/ 2
#Reorganize table for printing
schools_summary_f = schools_summary_table.rename(columns={"school_name":"School Name", 
                                                          "type":"School Type",
                                                          "Number of students":"Total Students",
                                                          "budget":"Total School Budget",
                                                          "Budget per Student":"Per Student Budget",
                                                          "Average math":"Average Math Score",
                                                          "Average read":"Average Reading Score",
                                                          "% passing math":"% Passing Math",
                                                          "% passing reading":"% Passing Reading",
                                                          "% overall passing rate":"Overall Passing Rate"})
schools_summary_table.set_index('school_name')

#Removing columns without use for final print
del schools_summary_f['math score']
del schools_summary_f['reading score']

#Formatting data
schools_summary_f["Total Students"] = schools_summary_f["Total Students"].map("{:,}".format)
schools_summary_f["Total School Budget"] = schools_summary_f["Total School Budget"].map("${:,}".format)
schools_summary_f


,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,"4,976","$3,124,928",628.0,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,"1,858","$1,081,356",582.0,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,"2,949","$1,884,411",639.0,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,"2,739","$1,763,916",644.0,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,"1,468","$917,500",625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Hernandez High School,District,"4,635","$3,022,020",652.0,77.289752,80.934412,66.752967,80.862999,73.807983
6,Holden High School,Charter,427,"$248,087",581.0,83.803279,83.814988,92.505855,96.252927,94.379391
7,Huang High School,District,"2,917","$1,910,635",655.0,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,"4,761","$3,094,650",650.0,77.072464,80.966394,66.057551,81.222432,73.639992
9,Pena High School,Charter,962,"$585,858",609.0,83.839917,84.044699,94.594595,95.945946,95.270270


In [6]:
#3-TOP PERFORMING SCHOOLS (By Passing Rate)
#Sorting top 5 performing schools based on Overall Passing Rate
schools_sum_sorted_t = schools_summary_table.sort_values("% overall passing rate", ascending=False).head()
schools_sum_sorted_t = schools_sum_sorted_t.reset_index(drop=True)

#Removing columns without use for final print
del schools_sum_sorted_t['math score']
del schools_sum_sorted_t['reading score']

#Reorganize table for printing
schools_sum_sorted_t = schools_sum_sorted_t.rename(columns={"school_name":"School Name", 
                                                          "type":"School Type",
                                                          "Number of students":"Total Students",
                                                          "budget":"Total School Budget",
                                                          "Budget per Student":"Per Student Budget",
                                                          "Average math":"Average Math Score",
                                                          "Average read":"Average Reading Score",
                                                          "% passing math":"% Passing Math",
                                                          "% passing reading":"% Passing Reading",
                                                          "% overall passing rate":"Overall Passing Rate"})
#Formatting data
schools_sum_sorted_t["Total Students"] = schools_sum_sorted_t["Total Students"].map("{:,}".format)
schools_sum_sorted_t["Total School Budget"] = schools_sum_sorted_t["Total School Budget"].map("${:,}".format)
schools_sum_sorted_t

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Cabrera High School,Charter,"1,858","$1,081,356",582.0,83.061895,83.975780,94.133477,97.039828,95.586652
1,Thomas High School,Charter,"1,635","$1,043,130",638.0,83.418349,83.848930,93.272171,97.308869,95.290520
2,Pena High School,Charter,962,"$585,858",609.0,83.839917,84.044699,94.594595,95.945946,95.270270
3,Griffin High School,Charter,"1,468","$917,500",625.0,83.351499,83.816757,93.392371,97.138965,95.265668
4,Wilson High School,Charter,"2,283","$1,319,574",578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [7]:
#4-BOTTON PERFORMING SCHOOLS (By Passing Rate)
#Sorting the bottom 5 performing schools based on Overall Passing Rate
schools_sum_sorted_b = schools_summary_table.sort_values("% overall passing rate").head()
schools_sum_sorted_b = schools_sum_sorted_b.reset_index(drop=True)

#Removing columns without use for final print
del schools_sum_sorted_b['math score']
del schools_sum_sorted_b['reading score']

#Reorganize table for printing
schools_sum_sorted_b = schools_sum_sorted_b.rename(columns={"school_name":"School Name", 
                                                          "type":"School Type",
                                                          "Number of students":"Total Students",
                                                          "budget":"Total School Budget",
                                                          "Budget per Student":"Per Student Budget",
                                                          "Average math":"Average Math Score",
                                                          "Average read":"Average Reading Score",
                                                          "% passing math":"% Passing Math",
                                                          "% passing reading":"% Passing Reading",
                                                          "% overall passing rate":"Overall Passing Rate"})
#Formatting data
schools_sum_sorted_b["Total Students"] = schools_sum_sorted_b["Total Students"].map("{:,}".format)
schools_sum_sorted_b["Total School Budget"] = schools_sum_sorted_b["Total School Budget"].map("${:,}".format)
schools_sum_sorted_b

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Rodriguez High School,District,"3,999","$2,547,363",637.0,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,District,"2,949","$1,884,411",639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Huang High School,District,"2,917","$1,910,635",655.0,76.629414,81.182722,65.683922,81.316421,73.500171
3,Johnson High School,District,"4,761","$3,094,650",650.0,77.072464,80.966394,66.057551,81.222432,73.639992
4,Ford High School,District,"2,739","$1,763,916",644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [8]:
#5-MATH SCORES BY GRADE
# Average math score for 9th, 10th, 11th, 12th grade
avg_mats_9 = school_data_c.loc[(school_data_c["grade"] == '9th'),:] 
avg_mats_10 = school_data_c.loc[(school_data_c["grade"] == '10th'),:] 
avg_mats_11 = school_data_c.loc[(school_data_c["grade"] == '11th'),:] 
avg_mats_12 = school_data_c.loc[(school_data_c["grade"] == '12th'),:] 

#Group the information to get it by school
avg_mats_9b = avg_mats_9.groupby(['school_name'])
avg_mats_10b = avg_mats_10.groupby(['school_name'])
avg_mats_11b = avg_mats_11.groupby(['school_name'])
avg_mats_12b = avg_mats_12.groupby(['school_name'])

#Calculate the average
avg_mats_9c = avg_mats_9b["math_score"].mean()
avg_mats_10c = avg_mats_10b["math_score"].mean()
avg_mats_11c = avg_mats_11b["math_score"].mean()
avg_mats_12c = avg_mats_12b["math_score"].mean()

#Conform the data frame with the results
math_scores = pd.DataFrame({"9th": avg_mats_9c,
                            "10th": avg_mats_10c,
                            "11th": avg_mats_11c,
                            "12th": avg_mats_12c})
math_scores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [9]:
#6-READING SCORES BY GRADE
# Average reading score for 9th, 10th, 11th, 12th grade
avg_read_9 = school_data_c.loc[(school_data_c["grade"] == '9th'),:] 
avg_read_10 = school_data_c.loc[(school_data_c["grade"] == '10th'),:] 
avg_read_11 = school_data_c.loc[(school_data_c["grade"] == '11th'),:] 
avg_read_12 = school_data_c.loc[(school_data_c["grade"] == '12th'),:] 

#Group the information to get it by school
avg_read_9b = avg_read_9.groupby(['school_name'])
avg_read_10b = avg_read_10.groupby(['school_name'])
avg_read_11b = avg_read_11.groupby(['school_name'])
avg_read_12b = avg_read_12.groupby(['school_name'])

#Calculate the average
avg_read_9c = avg_read_9b["reading_score"].mean()
avg_read_10c = avg_read_10b["reading_score"].mean()
avg_read_11c = avg_read_11b["reading_score"].mean()
avg_read_12c = avg_read_12b["reading_score"].mean()

#Conform the data frame with the results
reading_scores = pd.DataFrame({"9th": avg_read_9c,
                            "10th": avg_read_10c,
                            "11th": avg_read_11c,
                            "12th": avg_read_12c})
reading_scores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [10]:
#7-SCORES BY SCHOOL SPENDING
#Table that breaks down school performances based on average Spending Ranges (Per Student)
# Bins. 
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
spending_ranges = schools_summary_table
spending_ranges["Spending Ranges"]=pd.cut(spending_ranges["Budget per Student"], spending_bins, labels=group_names)
#Conforming the table
spending_ranges_f = spending_ranges[["Spending Ranges","Average math","Average read","% passing math","% passing reading", "% overall passing rate"]]
spending_ranges_g = spending_ranges_f.groupby("Spending Ranges")
spending_ranges_g.mean()

,Average math,Average read,% passing math,% passing reading,% overall passing rate
Spending Ranges,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [11]:
#8-SCORES BY SCHOOL SIZE
#Table that breaks down school performances based on school size
# Bins. 
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
shool_sizes = schools_summary_table
shool_sizes["School Size"]=pd.cut(shool_sizes["Number of students"], size_bins, labels=group_names)
#Conforming the table
shool_sizes_f = shool_sizes[["School Size","Average math","Average read","% passing math","% passing reading", "% overall passing rate"]]
shool_sizes_g = shool_sizes_f.groupby("School Size")
shool_sizes_g.mean()

,Average math,Average read,% passing math,% passing reading,% overall passing rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [12]:
#9-SCORES BY SCHOOL TYPE
#Table that breaks down school performances based on school type(here we do not use bins since the field is already on the table)
shool_types = schools_summary_table
#Conforming the table
shool_types_f = shool_types[["type","Average math","Average read","% passing math","% passing reading", "% overall passing rate"]]
shool_types_g = shool_types_f.groupby("type")
shool_types_g.mean()

,Average math,Average read,% passing math,% passing reading,% overall passing rate
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
